## You're ready to put into practice everything that you've learned so far. Here are the next steps for your capstone:







In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import linear_model

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from category_encoders import LeaveOneOutEncoder

from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest
from sklearn.svm import SVC

from category_encoders import LeaveOneOutEncoder

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion

from sqlalchemy import create_engine

import warnings

warnings.filterwarnings('ignore')

## 1) Go out and find a dataset of interest. It could be from one of the recommended resources or some other aggregation. Or it could be something that you scraped yourself. Just make sure that it has lots of variables, including an outcome of interest to you.

* Pro football reference game info

        https://www.pro-football-reference.com/years/{1966-2017}/games.htm/games.htm#games::none


* Odds - money line, open/close lines historic info

        http://www.aussportsbetting.com/historical_data/nfl.xlsx
    

* Kaggle - NFL scores and betting data

        https://www.kaggle.com/tobycrabtree/nfl-scores-and-betting-data
        
### Kaggle files:
       
* spreadspoke_scores.csv
* nfl_stadiums.csv
* nfl_teams.csv

In [2]:
all_dfs = []

# Read seasons 2000 to 2017
for year in range(2000,2018):
    print(year)
    url = "https://www.pro-football-reference.com/years/" + str(year) + "/games.htm#games::none"
    ydf = pd.read_html(url)
    ydf = ydf[0]
    ydf['Year'] = year

    all_dfs.append(ydf)
    
scores1_df = pd.concat(all_dfs)
scores1_df.shape

2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017


(5096, 15)

In [3]:
scores1_df.shape

(5096, 15)

In [4]:
odds_df = pd.read_excel("http://www.aussportsbetting.com/historical_data/nfl.xlsx")
scores2_df = pd.read_csv('data/spreadspoke_scores.csv', encoding = "ISO-8859-1", engine='python')
stadiums_df = pd.read_csv('data/nfl_stadiums.csv', encoding = "ISO-8859-1", engine='python')
teams_df = pd.read_csv('data/nfl_teams.csv', encoding = "ISO-8859-1", engine='python')

## 2) Explore the data. Get to know the data. Spend a lot of time going over its quirks. You should understand how it was gathered, what's in it, and what the variables look like.

In [5]:
scores1_df.head()

,Week,Day,Date,Time,Winner/tie,Unnamed: 5,Loser/tie,Unnamed: 7,PtsW,PtsL,YdsW,TOW,YdsL,TOL,Year
0,1,Sun,September 3,1:00PM,Minnesota Vikings,NaN,Chicago Bears,boxscore,30,27,374,1,425,1,2000
1,1,Sun,September 3,1:00PM,Indianapolis Colts,@,Kansas City Chiefs,boxscore,27,14,386,2,280,1,2000
2,1,Sun,September 3,1:01PM,Washington Redskins,NaN,Carolina Panthers,boxscore,20,17,396,0,236,1,2000
3,1,Sun,September 3,1:02PM,Atlanta Falcons,NaN,San Francisco 49ers,boxscore,36,28,359,1,339,1,2000
4,1,Sun,September 3,1:02PM,Baltimore Ravens,@,Pittsburgh Steelers,boxscore,16,0,336,0,223,1,2000


In [6]:
scores1_df.shape

(5096, 15)

In [7]:
scores1_df.isna().mean()

Week          0.003532
Day           0.003532
Date          0.000000
Time          0.003532
Winner/tie    0.003532
Unnamed: 5    0.596154
Loser/tie     0.003532
Unnamed: 7    0.060047
PtsW          0.003532
PtsL          0.003532
YdsW          0.003532
TOW           0.003532
YdsL          0.003532
TOL           0.003532
Year          0.000000
dtype: float64

In [8]:
scores1_df = scores1_df.drop(columns=['Unnamed: 5', 'Unnamed: 7', 'Time'])

In [9]:
scores1_df = scores1_df.dropna()

In [10]:
scores1_df['Date'].value_counts()

Date           288
December 24     76
November 23     51
December 28     50
December 23     49
              ... 
February 4       2
January 26       1
September 6      1
January 28       1
February 2       1
Name: Date, Length: 155, dtype: int64

In [11]:
mos = {'January': 1
       ,'February': 2
       ,'March': 3
       ,'April': 4
       ,'May': 5
       ,'June': 6
       ,'July': 7
       ,'August': 8
       ,'September': 9
       ,'October': 10
       ,'November': 11
       ,'December': 12}

In [12]:
match = 'January'
for m in mos:
    if m != 'January':
        match = match + '|' + m
scores1_df = scores1_df[scores1_df['Date'].str.contains(match)]

In [13]:
scores1_df['Month'] = scores1_df['Date'].str.split(' ').str[0]
scores1_df['DayofMonth'] = scores1_df['Date'].str.split(' ').str[1].astype(int)

scores1_df['month_num'] = scores1_df['Month'].map(mos)

In [14]:
scores1_df

,Week,Day,Date,Winner/tie,Loser/tie,PtsW,PtsL,YdsW,TOW,YdsL,TOL,Year,Month,DayofMonth,month_num
0,1,Sun,September 3,Minnesota Vikings,Chicago Bears,30,27,374,1,425,1,2000,September,3,9
1,1,Sun,September 3,Indianapolis Colts,Kansas City Chiefs,27,14,386,2,280,1,2000,September,3,9
2,1,Sun,September 3,Washington Redskins,Carolina Panthers,20,17,396,0,236,1,2000,September,3,9
3,1,Sun,September 3,Atlanta Falcons,San Francisco 49ers,36,28,359,1,339,1,2000,September,3,9
4,1,Sun,September 3,Baltimore Ravens,Pittsburgh Steelers,16,0,336,0,223,1,2000,September,3,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,Division,Sun,January 14,Jacksonville Jaguars,Pittsburgh Steelers,45,42,378,0,545,2,2017,January,14,1
280,Division,Sun,January 14,Minnesota Vikings,New Orleans Saints,29,24,403,1,358,2,2017,January,14,1
281,ConfChamp,Sun,January 21,New England Patriots,Jacksonville Jaguars,24,20,344,1,374,0,2017,January,21,1
282,ConfChamp,Sun,January 21,Philadelphia Eagles,Minnesota Vikings,38,7,456,0,333,3,2017,January,21,1


## --------------------------------

In [15]:
odds_df.head()

,Date,Home Team,Away Team,Home Score,Away Score,Overtime?,Playoff Game?,Neutral Venue?,Home Odds Open,Home Odds Min,...,Total Score Close,Total Score Over Open,Total Score Over Min,Total Score Over Max,Total Score Over Close,Total Score Under Open,Total Score Under Min,Total Score Under Max,Total Score Under Close,Notes
0,2020-09-14,Denver Broncos,Tennessee Titans,14,16,NaN,NaN,NaN,1.76,1.76,...,41.0,1.9,1.86,1.9,1.9,1.9,1.9,1.9,1.9,NaN
1,2020-09-14,New York Giants,Pittsburgh Steelers,16,26,NaN,NaN,NaN,2.60,2.60,...,45.0,1.9,1.90,1.9,1.9,1.9,1.9,1.9,1.9,NaN
2,2020-09-13,Los Angeles Rams,Dallas Cowboys,20,17,NaN,NaN,NaN,2.15,2.15,...,51.5,1.9,1.90,1.9,1.9,1.9,1.9,1.9,1.9,NaN
3,2020-09-13,New Orleans Saints,Tampa Bay Buccaneers,34,23,NaN,NaN,NaN,1.47,1.47,...,47.5,1.9,1.90,1.9,1.9,1.9,1.9,1.9,1.9,NaN
4,2020-09-13,San Francisco 49ers,Arizona Cardinals,20,24,NaN,NaN,NaN,1.28,1.28,...,48.0,1.9,1.90,1.9,1.9,1.9,1.9,1.9,1.9,NaN


In [16]:
odds_df.shape

(3754, 45)

In [17]:
odds_df.describe()

,Home Score,Away Score,Home Odds Open,Home Odds Min,Home Odds Max,Home Odds Close,Away Odds Open,Away Odds Min,Away Odds Max,Away Odds Close,...,Total Score Max,Total Score Close,Total Score Over Open,Total Score Over Min,Total Score Over Max,Total Score Over Close,Total Score Under Open,Total Score Under Min,Total Score Under Max,Total Score Under Close
count,3754.000000,3754.000000,3754.000000,1618.00000,1618.000000,1618.000000,3754.000000,1618.000000,1618.000000,1618.000000,...,1618.000000,1618.000000,1618.000000,1618.000000,1618.000000,1618.000000,1618.000000,1618.000000,1618.000000,1618.000000
mean,23.479489,21.238679,1.950855,1.83207,2.064315,1.954741,2.892580,2.573298,2.982897,2.780451,...,46.400494,45.441595,1.937429,1.903295,1.967795,1.933845,1.932285,1.924482,1.942302,1.936200
std,10.380618,10.035696,0.999993,0.79537,1.035032,0.912600,1.795221,1.342111,1.692837,1.560962,...,4.115823,4.115203,0.034949,0.033771,0.057890,0.041980,0.034153,0.033072,0.041815,0.043195
min,0.000000,0.000000,1.011111,1.02000,1.050000,1.020000,1.055556,1.030000,1.060000,1.050000,...,37.000000,35.000000,1.806000,1.740000,1.860000,1.740000,1.775000,1.830000,1.787000,1.813000
25%,16.000000,14.000000,1.340000,1.33300,1.416000,1.377000,1.695250,1.641000,1.813000,1.699000,...,43.500000,42.500000,1.900000,1.884000,1.900000,1.900000,1.900000,1.900000,1.900000,1.900000
50%,23.000000,21.000000,1.606061,1.57050,1.714000,1.640000,2.450000,2.250000,2.555000,2.400000,...,46.000000,45.000000,1.952000,1.900000,1.961000,1.925000,1.952000,1.909000,1.952000,1.925000
75%,30.000000,27.000000,2.250000,2.09000,2.397500,2.260000,3.480000,3.100000,3.560000,3.310000,...,49.000000,48.000000,1.952000,1.917000,2.010000,1.961000,1.952000,1.952000,1.970000,1.961000
max,62.000000,59.000000,12.000000,10.00000,15.500000,11.000000,31.000000,12.250000,16.000000,16.000000,...,63.500000,62.500000,2.170000,2.020000,2.170000,2.110000,2.120000,2.120000,2.090000,2.220000


In [18]:
odds_df.isna().mean()

Date                       0.000000
Home Team                  0.000000
Away Team                  0.000000
Home Score                 0.000000
Away Score                 0.000000
Overtime?                  0.941929
Playoff Game?              0.958977
Neutral Venue?             0.988013
Home Odds Open             0.000000
Home Odds Min              0.568993
Home Odds Max              0.568993
Home Odds Close            0.568993
Away Odds Open             0.000000
Away Odds Min              0.568993
Away Odds Max              0.568993
Away Odds Close            0.568993
Home Line Open             0.000000
Home Line Min              0.568993
Home Line Max              0.568993
Home Line Close            0.568993
Away Line Open             0.568993
Away Line Min              0.568993
Away Line Max              0.568993
Away Line Close            0.568993
Home Line Odds Open        0.568993
Home Line Odds Min         0.568993
Home Line Odds Max         0.568993
Home Line Odds Close       0

In [19]:
odds_df['Overtime?'].value_counts()

Y    218
Name: Overtime?, dtype: int64

In [20]:
odds_df['Playoff Game?'].value_counts()

Y    154
Name: Playoff Game?, dtype: int64

In [21]:
odds_df['Neutral Venue?'].value_counts()

Y    45
Name: Neutral Venue?, dtype: int64

In [22]:
odds_df['Overtime?'] = odds_df['Overtime?'].fillna('N')
odds_df['Playoff Game?'] = odds_df['Playoff Game?'].fillna('N')
odds_df['Playoff Game?'].value_counts()

N    3600
Y     154
Name: Playoff Game?, dtype: int64

In [23]:
odds_cols = ['Date'                       
            ,'Home Team'                 
            ,'Away Team'                 
            ,'Home Score'                
            ,'Away Score'               
            ,'Overtime?'               
            ,'Playoff Game?'                       
            ,'Home Odds Open'             
            ,'Away Odds Open'           
            ,'Total Score Open']         
odds_df = odds_df[odds_cols]

In [24]:
odds_df['date'] = pd.to_datetime(odds_df['Date'])
odds_df['year'] = odds_df['date'].dt.year
odds_df['month'] = odds_df['date'].dt.month
odds_df['DayofMonth'] = odds_df['date'].dt.day

In [25]:
odds_df

,Date,Home Team,Away Team,Home Score,Away Score,Overtime?,Playoff Game?,Home Odds Open,Away Odds Open,Total Score Open,date,year,month,DayofMonth
0,2020-09-14,Denver Broncos,Tennessee Titans,14,16,N,N,1.760000,2.100000,42.0,2020-09-14,2020,9,14
1,2020-09-14,New York Giants,Pittsburgh Steelers,16,26,N,N,2.600000,1.550000,48.0,2020-09-14,2020,9,14
2,2020-09-13,Los Angeles Rams,Dallas Cowboys,20,17,N,N,2.150000,1.740000,50.0,2020-09-13,2020,9,13
3,2020-09-13,New Orleans Saints,Tampa Bay Buccaneers,34,23,N,N,1.470000,2.750000,49.5,2020-09-13,2020,9,13
4,2020-09-13,San Francisco 49ers,Arizona Cardinals,20,24,N,N,1.280000,3.750000,46.0,2020-09-13,2020,9,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3749,2006-09-10,Detroit Lions,Seattle Seahawks,6,9,N,N,3.500000,1.333333,44.0,2006-09-10,2006,9,10
3750,2006-09-10,Cleveland Browns,New Orleans Saints,14,19,N,N,1.500000,2.750000,36.5,2006-09-10,2006,9,10
3751,2006-09-10,Carolina Panthers,Atlanta Falcons,6,20,N,N,1.434783,2.900000,38.5,2006-09-10,2006,9,10
3752,2006-09-10,Arizona Cardinals,San Francisco 49ers,34,27,N,N,1.181818,5.000000,43.5,2006-09-10,2006,9,10


## --------------------------------

In [26]:
scores2_df.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,9/2/1966,1966,1,False,Miami Dolphins,14,23,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71,NaN
1,9/3/1966,1966,1,False,Houston Oilers,45,7,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70,NaN
2,9/4/1966,1966,1,False,San Diego Chargers,27,7,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82,NaN
3,9/9/1966,1966,2,False,Miami Dolphins,14,19,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78,NaN
4,9/10/1966,1966,1,False,Green Bay Packers,24,3,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62,NaN


In [27]:
scores2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12678 entries, 0 to 12677
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   schedule_date        12678 non-null  object 
 1   schedule_season      12678 non-null  int64  
 2   schedule_week        12678 non-null  object 
 3   schedule_playoff     12678 non-null  bool   
 4   team_home            12678 non-null  object 
 5   score_home           12678 non-null  int64  
 6   score_away           12678 non-null  int64  
 7   team_away            12678 non-null  object 
 8   team_favorite_id     10199 non-null  object 
 9   spread_favorite      10199 non-null  float64
 10  over_under_line      10189 non-null  object 
 11  stadium              12678 non-null  object 
 12  stadium_neutral      12678 non-null  bool   
 13  weather_temperature  11936 non-null  float64
 14  weather_wind_mph     11936 non-null  float64
 15  weather_humidity     8388 non-null  

In [28]:
scores2_df.isna().mean()

schedule_date          0.000000
schedule_season        0.000000
schedule_week          0.000000
schedule_playoff       0.000000
team_home              0.000000
score_home             0.000000
score_away             0.000000
team_away              0.000000
team_favorite_id       0.195536
spread_favorite        0.195536
over_under_line        0.196324
stadium                0.000000
stadium_neutral        0.000000
weather_temperature    0.058527
weather_wind_mph       0.058527
weather_humidity       0.338381
weather_detail         0.791844
dtype: float64

In [29]:
scores2_df = scores2_df.drop(columns = ['weather_humidity', 'weather_detail', 'schedule_playoff'])

In [30]:
scores2_df = scores2_df.dropna()

In [31]:
scores2_df['DayofMonth'] = pd.to_datetime(scores2_df['schedule_date']).dt.day
scores2_df['Month'] = pd.to_datetime(scores2_df['schedule_date']).dt.month
scores2_df['year'] = pd.to_datetime(scores2_df['schedule_date']).dt.year

### Add winning team column based on scores

In [32]:
scores2_df['Winner'] = scores2_df['team_home']

scores2_df.loc[scores2_df['score_away'] > scores2_df['score_home'], 
               "Winner"] = scores2_df['team_away']

In [33]:
scores2_df

,schedule_date,schedule_season,schedule_week,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,DayofMonth,Month,year,Winner
350,1/14/1968,1967,Superbowl,Green Bay Packers,33,14,Oakland Raiders,GB,-13.5,43,Orange Bowl,True,60.0,12.0,14,1,1968,Green Bay Packers
538,1/12/1969,1968,Superbowl,Baltimore Colts,7,16,New York Jets,IND,-18.0,40,Orange Bowl,True,66.0,12.0,12,1,1969,New York Jets
727,1/11/1970,1969,Superbowl,Kansas City Chiefs,23,7,Minnesota Vikings,MIN,-12.0,39,Tulane Stadium,True,55.0,14.0,11,1,1970,Kansas City Chiefs
916,1/17/1971,1970,Superbowl,Baltimore Colts,16,13,Dallas Cowboys,IND,-2.5,36,Orange Bowl,True,59.0,11.0,17,1,1971,Baltimore Colts
1105,1/16/1972,1971,Superbowl,Dallas Cowboys,24,3,Miami Dolphins,DAL,-6.0,34,Tulane Stadium,True,34.0,18.0,16,1,1972,Dallas Cowboys
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12655,12/29/2019,2019,17,Dallas Cowboys,47,16,Washington Redskins,DAL,-12.5,47.5,AT&T Stadium,False,72.0,0.0,29,12,2019,Dallas Cowboys
12657,12/29/2019,2019,17,Detroit Lions,20,23,Green Bay Packers,GB,-13.0,44,Ford Field,False,72.0,0.0,29,12,2019,Green Bay Packers
12658,12/29/2019,2019,17,Houston Texans,14,35,Tennessee Titans,TEN,-10.0,43,NRG Stadium,False,72.0,0.0,29,12,2019,Tennessee Titans
12667,1/4/2020,2019,Wildcard,Houston Texans,22,19,Buffalo Bills,HOU,-2.5,43,NRG Stadium,False,72.0,0.0,4,1,2020,Houston Texans


## --------------------------------

In [34]:
stadiums_df.head()

,stadium_name,stadium_location,stadium_open,stadium_close,stadium_type,stadium_address,stadium_weather_station_code,stadium_weather_type,stadium_capacity,stadium_surface,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION
0,Alamo Dome,"San Antonio, TX",NaN,NaN,indoor,"100 Montana St, San Antonio, TX 78203",78203,dome,72000,FieldTurf,NaN,NaN,NaN,NaN,NaN
1,Alltel Stadium,"Jacksonville, FL",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alumni Stadium,"Chestnut Hill, MA",NaN,NaN,outdoor,"Perimeter Rd, Chestnut Hill, MA 02467",2467,cold,NaN,Grass,NaN,NaN,NaN,NaN,NaN
3,Anaheim Stadium,"Anaheim, CA",1980.0,1994.0,outdoor,"2000 E Gene Autry Way, Anaheim, CA 92806",92806,warm,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arrowhead Stadium,"Kansas City, MO",1972.0,NaN,outdoor,"1 Arrowhead Dr, Kansas City, MO 64129",64129,cold,"76,416",Grass,US1MOJC0028,"KANSAS CITY 5.1 SE, MO US",39.0692,-94.4871,264.9


In [35]:
stadiums_df.isna().mean()

stadium_name                    0.00
stadium_location                0.00
stadium_open                    0.24
stadium_close                   0.59
stadium_type                    0.07
stadium_address                 0.09
stadium_weather_station_code    0.10
stadium_weather_type            0.07
stadium_capacity                0.61
stadium_surface                 0.47
STATION                         0.48
NAME                            0.48
LATITUDE                        0.48
LONGITUDE                       0.48
ELEVATION                       0.48
dtype: float64

In [36]:
cols = ['stadium_name', 'stadium_location', 'stadium_type', 'stadium_weather_type', 'stadium_surface']
stadiums_df = stadiums_df[cols]

In [37]:
stadiums_df.isna().mean()

stadium_name            0.00
stadium_location        0.00
stadium_type            0.07
stadium_weather_type    0.07
stadium_surface         0.47
dtype: float64

In [38]:
stadiums_df['stadium_type'] = stadiums_df['stadium_type'].fillna('unknown')
stadiums_df['stadium_weather_type'] = stadiums_df['stadium_weather_type'].fillna('unknown')
stadiums_df['stadium_surface'] = stadiums_df['stadium_surface'].fillna('unknown')

In [39]:
stadiums_df.isna().mean()

stadium_name            0.0
stadium_location        0.0
stadium_type            0.0
stadium_weather_type    0.0
stadium_surface         0.0
dtype: float64

In [40]:
stadiums_df

,stadium_name,stadium_location,stadium_type,stadium_weather_type,stadium_surface
0,Alamo Dome,"San Antonio, TX",indoor,dome,FieldTurf
1,Alltel Stadium,"Jacksonville, FL",unknown,unknown,unknown
2,Alumni Stadium,"Chestnut Hill, MA",outdoor,cold,Grass
3,Anaheim Stadium,"Anaheim, CA",outdoor,warm,unknown
4,Arrowhead Stadium,"Kansas City, MO",outdoor,cold,Grass
...,...,...,...,...,...
95,War Memorial Stadium,"Buffalo, NY",outdoor,cold,unknown
96,Wembley Stadium,"London, UK",outdoor,moderate,unknown
97,Wrigley Field,"Chicago, IL",outdoor,cold,unknown
98,Yale Bowl,"New Haven, CT",outdoor,cold,Grass


## --------------------------------

In [41]:
teams_df.head(45)

,team_name,team_name_short,team_id,team_id_pfr,team_conference,team_division,team_conference_pre2002,team_division_pre2002
0,Arizona Cardinals,Cardinals,ARI,CRD,NFC,NFC West,NFC,NFC West
1,Phoenix Cardinals,Cardinals,ARI,CRD,NFC,NaN,NFC,NFC East
2,St. Louis Cardinals,Cardinals,ARI,ARI,NFC,NaN,NFC,NFC East
3,Atlanta Falcons,Falcons,ATL,ATL,NFC,NFC South,NFC,NFC West
4,Baltimore Ravens,Ravens,BAL,RAV,AFC,AFC North,AFC,AFC Central
5,Buffalo Bills,Bills,BUF,BUF,AFC,AFC East,AFC,AFC East
6,Carolina Panthers,Panthers,CAR,CAR,NFC,NFC South,NFC,NFC West
7,Chicago Bears,Bears,CHI,CHI,NFC,NFC North,NFC,NFC Central
8,Cincinnati Bengals,Bengals,CIN,CIN,AFC,AFC North,AFC,AFC Central
9,Cleveland Browns,Browns,CLE,CLE,AFC,AFC North,AFC,AFC Central


In [42]:
teams_df.shape

(41, 8)

In [43]:
teams_df.isna().mean()

team_name                  0.000000
team_name_short            0.000000
team_id                    0.000000
team_id_pfr                0.000000
team_conference            0.000000
team_division              0.195122
team_conference_pre2002    0.000000
team_division_pre2002      0.048780
dtype: float64

In [44]:
teams_df['team_division'] = teams_df['team_division'].fillna(teams_df['team_division_pre2002'])
teams_df['team_division'] = teams_df['team_division'].fillna('unknown')

In [45]:
teams_df = teams_df.drop(columns=['team_conference_pre2002', 'team_division_pre2002', 'team_id_pfr'])

In [46]:
teams_df

,team_name,team_name_short,team_id,team_conference,team_division
0,Arizona Cardinals,Cardinals,ARI,NFC,NFC West
1,Phoenix Cardinals,Cardinals,ARI,NFC,NFC East
2,St. Louis Cardinals,Cardinals,ARI,NFC,NFC East
3,Atlanta Falcons,Falcons,ATL,NFC,NFC South
4,Baltimore Ravens,Ravens,BAL,AFC,AFC North
5,Buffalo Bills,Bills,BUF,AFC,AFC East
6,Carolina Panthers,Panthers,CAR,NFC,NFC South
7,Chicago Bears,Bears,CHI,NFC,NFC North
8,Cincinnati Bengals,Bengals,CIN,AFC,AFC North
9,Cleveland Browns,Browns,CLE,AFC,AFC North


## Join dataframes together

In [47]:
scores1_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4790 entries, 0 to 283
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Week        4790 non-null   object
 1   Day         4790 non-null   object
 2   Date        4790 non-null   object
 3   Winner/tie  4790 non-null   object
 4   Loser/tie   4790 non-null   object
 5   PtsW        4790 non-null   object
 6   PtsL        4790 non-null   object
 7   YdsW        4790 non-null   object
 8   TOW         4790 non-null   object
 9   YdsL        4790 non-null   object
 10  TOL         4790 non-null   object
 11  Year        4790 non-null   int64 
 12  Month       4790 non-null   object
 13  DayofMonth  4790 non-null   int32 
 14  month_num   4790 non-null   int64 
dtypes: int32(1), int64(2), object(12)
memory usage: 580.0+ KB


In [48]:
scores2_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9677 entries, 350 to 12669
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   schedule_date        9677 non-null   object 
 1   schedule_season      9677 non-null   int64  
 2   schedule_week        9677 non-null   object 
 3   team_home            9677 non-null   object 
 4   score_home           9677 non-null   int64  
 5   score_away           9677 non-null   int64  
 6   team_away            9677 non-null   object 
 7   team_favorite_id     9677 non-null   object 
 8   spread_favorite      9677 non-null   float64
 9   over_under_line      9677 non-null   object 
 10  stadium              9677 non-null   object 
 11  stadium_neutral      9677 non-null   bool   
 12  weather_temperature  9677 non-null   float64
 13  weather_wind_mph     9677 non-null   float64
 14  DayofMonth           9677 non-null   int64  
 15  Month                9677 non-null 

In [49]:
full_table = scores1_df.merge(scores2_df,
                              left_on=['month_num', 'DayofMonth', 'Year', 'Winner/tie'], 
                              right_on=['Month', 'DayofMonth', 'year', 'Winner'])

In [50]:
full_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4467 entries, 0 to 4466
Data columns (total 32 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Week                 4467 non-null   object 
 1   Day                  4467 non-null   object 
 2   Date                 4467 non-null   object 
 3   Winner/tie           4467 non-null   object 
 4   Loser/tie            4467 non-null   object 
 5   PtsW                 4467 non-null   object 
 6   PtsL                 4467 non-null   object 
 7   YdsW                 4467 non-null   object 
 8   TOW                  4467 non-null   object 
 9   YdsL                 4467 non-null   object 
 10  TOL                  4467 non-null   object 
 11  Year                 4467 non-null   int64  
 12  Month_x              4467 non-null   object 
 13  DayofMonth           4467 non-null   int32  
 14  month_num            4467 non-null   int64  
 15  schedule_date        4467 non-null   o

In [51]:
full_table[['Date', 'Winner/tie', 'Winner', 'PtsW', 'PtsL', 'score_home', 'score_away']]

,Date,Winner/tie,Winner,PtsW,PtsL,score_home,score_away
0,September 3,Minnesota Vikings,Minnesota Vikings,30,27,30,27
1,September 3,Indianapolis Colts,Indianapolis Colts,27,14,14,27
2,September 3,Washington Redskins,Washington Redskins,20,17,20,17
3,September 3,Atlanta Falcons,Atlanta Falcons,36,28,36,28
4,September 3,Baltimore Ravens,Baltimore Ravens,16,0,0,16
...,...,...,...,...,...,...,...
4462,December 31,Cincinnati Bengals,Cincinnati Bengals,31,27,27,31
4463,December 31,Atlanta Falcons,Atlanta Falcons,22,10,22,10
4464,December 31,San Francisco 49ers,San Francisco 49ers,34,13,13,34
4465,December 31,Arizona Cardinals,Arizona Cardinals,26,24,24,26


In [52]:
full_table = full_table.merge(odds_df,
                              left_on=['month_num', 'DayofMonth', 'year', 'team_home'], 
                              right_on=['month', 'DayofMonth', 'year', 'Home Team'])

In [53]:
# Get team information from team ID
full_table = full_table.merge(teams_df,
                              left_on=['team_favorite_id'], 
                              right_on=['team_id'])

In [54]:
# Remove unneeded rows
full_table = full_table.drop(columns=['Date_x', 'Month_x'
                                        ,'schedule_date'
                                        ,'Week'
                                        ,'Month_y'
                                        ,'year'
                                        ,'Winner'
                                        ,'Home Score'
                                        ,'Away Score'
                                        ,'Home Team'
                                        ,'Away Team'
                                        ,'Date_y'
                                        ,'team_name_short'
                                        ,'team_id'
                                        ,'month_num'])

In [55]:
# Get stadium information
full_table = full_table.merge(stadiums_df,
                              left_on=['stadium'], 
                              right_on=['stadium_name'])

In [56]:
# Rename columns
full_table.rename(columns = {'Day':'weekday'
                             ,'Winner/tie':'winner'
                             ,'Loser/tie':'loser'
                             ,'PtsW':'pts_win' 
                             ,'PtsL':'pts_lose'
                             ,'YdsW':'yds_win' 
                             ,'YdsL':'yds_lose'
                             ,'TOW':'TO_win' 
                             ,'TOL':'TO_lose'
                             ,'Year':'year'
                             ,'weather_temperature':'w_temp'
                             ,'weather_wind_mph':'w_wind_mph'
                             ,'Overtime?':'overtime'
                             ,'Playoff Game?':'playoff'
                             ,'Home Odds Open':'home_odds_open'
                             ,'Away Odds Open':'away_odds_open'
                             ,'Total Score Open':'total_score_open'
                             ,'team_name':'team_fav_name'
                             ,'team_conference':'team_fav_conf'
                             ,'team_division':'team_fav_div'
                             }
                             , inplace = True)

In [57]:
# Get yards, points, turnover differences
full_table['yds_diff'] = full_table['yds_win'].astype(int) - full_table['yds_lose'].astype(int)
full_table['pts_diff'] = full_table['pts_win'].astype(int) - full_table['pts_lose'].astype(int)
full_table['TO_diff'] = full_table['TO_win'].astype(int) - full_table['TO_lose'].astype(int)

In [58]:
# Remove the difference rows
full_table = full_table.drop(columns=['yds_win', 'yds_lose', 'pts_win', 'pts_lose', 
                                      'TO_win', 'TO_lose'])

In [59]:
full_table['schedule_week'].value_counts()

15          246
1           243
12          241
16          235
2           233
3           232
13          230
14          228
5           214
11          212
4           211
10          209
8           208
7           206
6           197
9           196
17          143
Division      1
Wildcard      1
Name: schedule_week, dtype: int64

In [60]:
list = ''
for i in range(1, 18):
    if i == 17: 
        list = list + str(i)
    else:
        list = list + str(i) + '|'
    
full_table = full_table[full_table['schedule_week'].str.contains(list)]

In [61]:
# Convert certain rows to integers
full_table['schedule_week'] = full_table['schedule_week'].astype(str).astype(int)

full_table['spread_favorite'] = full_table['over_under_line'].astype(str).astype(float).astype(int)

full_table['w_temp'] = full_table['w_temp'].astype(int)
full_table['w_wind_mph'] = full_table['w_wind_mph'].astype(int)

## 3) Model your outcome of interest. You should try several different approaches and really work to tune a variety of models before using the model evaluation techniques to choose what you consider to be the best performer. Make sure to think about explanatory versus predictive power, and experiment with both.

In [62]:
full_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3684 entries, 0 to 3685
Data columns (total 36 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   weekday               3684 non-null   object        
 1   winner                3684 non-null   object        
 2   loser                 3684 non-null   object        
 3   year                  3684 non-null   int64         
 4   DayofMonth            3684 non-null   int32         
 5   schedule_season       3684 non-null   int64         
 6   schedule_week         3684 non-null   int32         
 7   team_home             3684 non-null   object        
 8   score_home            3684 non-null   int64         
 9   score_away            3684 non-null   int64         
 10  team_away             3684 non-null   object        
 11  team_favorite_id      3684 non-null   object        
 12  spread_favorite       3684 non-null   int32         
 13  over_under_line   

In [63]:
cols = full_table.columns
cols

Index(['weekday', 'winner', 'loser', 'year', 'DayofMonth', 'schedule_season',
       'schedule_week', 'team_home', 'score_home', 'score_away', 'team_away',
       'team_favorite_id', 'spread_favorite', 'over_under_line', 'stadium',
       'stadium_neutral', 'w_temp', 'w_wind_mph', 'overtime', 'playoff',
       'home_odds_open', 'away_odds_open', 'total_score_open', 'date', 'month',
       'team_fav_name', 'team_fav_conf', 'team_fav_div', 'stadium_name',
       'stadium_location', 'stadium_type', 'stadium_weather_type',
       'stadium_surface', 'yds_diff', 'pts_diff', 'TO_diff'],
      dtype='object')

In [64]:
num_cols = ['year', 'DayofMonth', 'schedule_week', 'schedule_season', 'score_home', 'score_away', 
            'spread_favorite', 'over_under_line', 'w_temp', 'w_wind_mph','home_odds_open', 'away_odds_open', 
            'total_score_open','month', 'yds_diff', 'pts_diff', 'TO_diff']

In [65]:
bin_cols = ['stadium_neutral', 'overtime']

new_cols = []
for col in bin_cols:
    new_col = 'd_' + col
    full_table[new_col] = pd.get_dummies(full_table[col], drop_first=True)
    full_table[new_col] = full_table[new_col].astype(int)
    new_cols.append(new_col)
    
    # drop the old columns
    full_table = full_table.drop(columns=[col])
    
bin_cols = new_cols

In [66]:
cat_cols = ['weekday'      
            ,'winner'     
            ,'loser'  
            ,'team_home'
            ,'team_away'
            ,'stadium'
            ,'team_fav_name'   
            ,'team_fav_conf'
            ,'team_fav_div'
            ,'stadium_name'
            ,'stadium_location'
            ,'stadium_type'
            ,'stadium_weather_type'
            ,'stadium_surface']

new_cols = []
for col in cat_cols:
    new_col = 'd_' + col
    full_table[col] = pd.Categorical(full_table[col])
    full_table[new_col] = full_table[col].cat.codes
    new_cols.append(new_col)
    
    # drop the old columns
#     if (col not in ['winner', 'loser']):
#         full_table = full_table.drop(columns=[col])
    
cat_cols = new_cols

In [67]:
full_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3684 entries, 0 to 3685
Data columns (total 50 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   weekday                 3684 non-null   category      
 1   winner                  3684 non-null   category      
 2   loser                   3684 non-null   category      
 3   year                    3684 non-null   int64         
 4   DayofMonth              3684 non-null   int32         
 5   schedule_season         3684 non-null   int64         
 6   schedule_week           3684 non-null   int32         
 7   team_home               3684 non-null   category      
 8   score_home              3684 non-null   int64         
 9   score_away              3684 non-null   int64         
 10  team_away               3684 non-null   category      
 11  team_favorite_id        3684 non-null   object        
 12  spread_favorite         3684 non-null   int32   

## Target: predict if home team will win based on previous 2 games

In [68]:
full_table.columns

Index(['weekday', 'winner', 'loser', 'year', 'DayofMonth', 'schedule_season',
       'schedule_week', 'team_home', 'score_home', 'score_away', 'team_away',
       'team_favorite_id', 'spread_favorite', 'over_under_line', 'stadium',
       'w_temp', 'w_wind_mph', 'playoff', 'home_odds_open', 'away_odds_open',
       'total_score_open', 'date', 'month', 'team_fav_name', 'team_fav_conf',
       'team_fav_div', 'stadium_name', 'stadium_location', 'stadium_type',
       'stadium_weather_type', 'stadium_surface', 'yds_diff', 'pts_diff',
       'TO_diff', 'd_stadium_neutral', 'd_overtime', 'd_weekday', 'd_winner',
       'd_loser', 'd_team_home', 'd_team_away', 'd_stadium', 'd_team_fav_name',
       'd_team_fav_conf', 'd_team_fav_div', 'd_stadium_name',
       'd_stadium_location', 'd_stadium_type', 'd_stadium_weather_type',
       'd_stadium_surface'],
      dtype='object')

In [70]:
winner_df = full_table
winner_df['game'] = winner_df['schedule_season'].astype(str) + winner_df['schedule_week'].astype(str) + winner_df['d_team_home'].astype(str)

In [71]:
winner_df

,weekday,winner,loser,year,DayofMonth,schedule_season,schedule_week,team_home,score_home,score_away,...,d_stadium,d_team_fav_name,d_team_fav_conf,d_team_fav_div,d_stadium_name,d_stadium_location,d_stadium_type,d_stadium_weather_type,d_stadium_surface,game
0,Thu,Pittsburgh Steelers,Miami Dolphins,2006,7,2006,1,Pittsburgh Steelers,28,17,...,15,31,0,2,15,28,1,0,1,2006126
1,Sun,Cincinnati Bengals,Pittsburgh Steelers,2006,24,2006,3,Pittsburgh Steelers,20,28,...,15,31,0,2,15,28,1,0,1,2006326
2,Sun,Pittsburgh Steelers,Kansas City Chiefs,2006,15,2006,6,Pittsburgh Steelers,45,7,...,15,31,0,2,15,28,1,0,1,2006626
3,Sun,Denver Broncos,Pittsburgh Steelers,2006,5,2006,9,Pittsburgh Steelers,20,31,...,15,31,0,2,15,28,1,0,1,2006926
4,Sun,Pittsburgh Steelers,New Orleans Saints,2006,12,2006,10,Pittsburgh Steelers,38,31,...,15,31,0,2,15,28,1,0,1,20061026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3681,Thu,New York Jets,Buffalo Bills,2009,3,2009,13,Buffalo Bills,13,19,...,38,27,1,1,38,35,0,1,2,2009133
3682,Sun,Minnesota Vikings,Cleveland Browns,2017,29,2017,8,Cleveland Browns,16,33,...,45,23,1,6,45,18,1,2,2,201787
3683,Sun,Los Angeles Rams,Arizona Cardinals,2017,22,2017,7,Los Angeles Rams,33,0,...,45,21,1,8,45,18,1,2,2,2017717
3684,Sun,Los Angeles Rams,Arizona Cardinals,2017,22,2017,7,Los Angeles Rams,33,0,...,45,36,1,8,45,18,1,2,2,2017717


In [72]:
#winner_df = winner_df.groupby(['schedule_season', 'schedule_week', 'team_home'])
winner_df['prev1_winner'] = winner_df.groupby('game')['winner'].shift(1)
#winner_df['prev2_winner'] = winner_df.groupby(['schedule_season', 'schedule_week', 'winner'])['winner'].shift(2)
winner_df[['schedule_season', 'schedule_week', 'team_home', 'prev1_winner']]

,schedule_season,schedule_week,team_home,prev1_winner
0,2006,1,Pittsburgh Steelers,NaN
1,2006,3,Pittsburgh Steelers,NaN
2,2006,6,Pittsburgh Steelers,NaN
3,2006,9,Pittsburgh Steelers,NaN
4,2006,10,Pittsburgh Steelers,NaN
...,...,...,...,...
3681,2009,13,Buffalo Bills,NaN
3682,2017,8,Cleveland Browns,NaN
3683,2017,7,Los Angeles Rams,NaN
3684,2017,7,Los Angeles Rams,Los Angeles Rams


In [73]:
# winner_df['prev1_winner'] = winner_df['prev1_winner'].cat.add_categories('lost-game')
# winner_df['prev1_winner'].fillna('lost-game', inplace=True)

# winner_df['prev2_winner'] = winner_df['prev2_winner'].cat.add_categories('lost-game')
# winner_df['prev2_winner'].fillna('lost-game', inplace=True)

In [74]:
winner_df['prev1_winner'].value_counts()

New England Patriots    138
Arizona Cardinals       125
Tennessee Titans        110
Indianapolis Colts       98
San Diego Chargers       76
Oakland Raiders          32
St. Louis Rams           31
Jacksonville Jaguars     21
Miami Dolphins           18
Kansas City Chiefs       17
Houston Texans           15
San Francisco 49ers      15
Pittsburgh Steelers      14
New York Jets            14
Atlanta Falcons          13
Denver Broncos           12
Los Angeles Rams         12
Carolina Panthers        11
Buffalo Bills            10
Baltimore Ravens         10
Seattle Seahawks          9
Cincinnati Bengals        9
Chicago Bears             8
Cleveland Browns          8
Los Angeles Chargers      7
Detroit Lions             7
Dallas Cowboys            6
Minnesota Vikings         6
New Orleans Saints        6
Philadelphia Eagles       6
New York Giants           5
Tampa Bay Buccaneers      5
Green Bay Packers         3
Washington Redskins       3
Name: prev1_winner, dtype: int64

In [75]:
#winner_df['prev2_winner'].value_counts()

In [76]:
winner_df[(winner_df['schedule_season'] == 2017) & (winner_df['schedule_week'] == 5)]

,weekday,winner,loser,year,DayofMonth,schedule_season,schedule_week,team_home,score_home,score_away,...,d_team_fav_name,d_team_fav_conf,d_team_fav_div,d_stadium_name,d_stadium_location,d_stadium_type,d_stadium_weather_type,d_stadium_surface,game,prev1_winner
77,Sun,Jacksonville Jaguars,Pittsburgh Steelers,2017,8,2017,5,Pittsburgh Steelers,9,30,...,31,0,2,15,28,1,0,1,2017526,NaN
446,Sun,Baltimore Ravens,Oakland Raiders,2017,8,2017,5,Oakland Raiders,17,30,...,20,0,4,31,25,1,2,1,2017524,NaN
447,Sun,Baltimore Ravens,Oakland Raiders,2017,8,2017,5,Oakland Raiders,17,30,...,28,0,4,31,25,1,2,1,2017524,Baltimore Ravens
555,Sun,New York Jets,Cleveland Browns,2017,8,2017,5,Cleveland Browns,14,17,...,27,1,1,9,6,1,0,1,201757,NaN
1020,Sun,Cincinnati Bengals,Buffalo Bills,2017,8,2017,5,Cincinnati Bengals,20,16,...,8,0,2,32,5,1,0,0,201756,NaN
1624,Mon,Minnesota Vikings,Chicago Bears,2017,9,2017,5,Chicago Bears,17,20,...,23,1,6,39,4,1,0,1,201755,NaN
1732,Sun,Carolina Panthers,Detroit Lions,2017,8,2017,5,Detroit Lions,24,27,...,12,1,6,10,8,0,1,0,2017510,NaN
1908,Thu,New England Patriots,Tampa Bay Buccaneers,2017,5,2017,5,Tampa Bay Buccaneers,14,19,...,24,0,1,36,34,1,3,1,2017531,NaN
1909,Thu,New England Patriots,Tampa Bay Buccaneers,2017,5,2017,5,Tampa Bay Buccaneers,14,19,...,4,0,9,36,34,1,3,1,2017531,New England Patriots
2249,Sun,Indianapolis Colts,San Francisco 49ers,2017,8,2017,5,Indianapolis Colts,26,23,...,2,0,1,23,14,2,1,0,2017513,NaN


In [77]:
winner_df[(winner_df['schedule_season'] == 2017) & (winner_df['schedule_week'] == 4)]

,weekday,winner,loser,year,DayofMonth,schedule_season,schedule_week,team_home,score_home,score_away,...,d_team_fav_name,d_team_fav_conf,d_team_fav_div,d_stadium_name,d_stadium_location,d_stadium_type,d_stadium_weather_type,d_stadium_surface,game,prev1_winner
300,Sun,Buffalo Bills,Atlanta Falcons,2017,1,2017,4,Atlanta Falcons,17,23,...,1,1,7,11,1,0,1,0,201741,NaN
469,Sun,Cincinnati Bengals,Cleveland Browns,2017,1,2017,4,Cleveland Browns,7,31,...,8,0,2,9,6,1,0,1,201747,NaN
831,Sun,Arizona Cardinals,San Francisco 49ers,2017,1,2017,4,Arizona Cardinals,18,15,...,0,1,8,47,11,2,1,1,201740,NaN
832,Sun,Arizona Cardinals,San Francisco 49ers,2017,1,2017,4,Arizona Cardinals,18,15,...,30,1,5,47,11,2,1,1,201740,Arizona Cardinals
833,Sun,Arizona Cardinals,San Francisco 49ers,2017,1,2017,4,Arizona Cardinals,18,15,...,35,1,5,47,11,2,1,1,201740,Arizona Cardinals
929,Sun,Denver Broncos,Oakland Raiders,2017,1,2017,4,Denver Broncos,16,10,...,11,0,4,40,7,1,0,1,201749,NaN
1243,Sun,Pittsburgh Steelers,Baltimore Ravens,2017,1,2017,4,Baltimore Ravens,9,26,...,31,0,2,24,2,1,2,0,201742,NaN
1847,Mon,Kansas City Chiefs,Washington Redskins,2017,2,2017,4,Kansas City Chiefs,29,20,...,18,0,4,1,17,1,0,1,2017415,NaN
1905,Sun,Tampa Bay Buccaneers,New York Giants,2017,1,2017,4,Tampa Bay Buccaneers,25,23,...,37,1,7,36,34,1,3,1,2017431,NaN
2392,Sun,New York Jets,Jacksonville Jaguars,2017,1,2017,4,New York Jets,23,20,...,17,0,3,27,9,1,0,0,2017423,NaN


## Target: points difference based on previous 2 games

https://stackoverflow.com/questions/53335567/use-pandas-shift-within-a-group/53335744#53335744

## Histograms

In [ ]:
numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']

numeric_df = full_table.select_dtypes(include=numerics)
numeric_df.hist(bins=30, figsize=(15, 15))
plt.show()

In [ ]:
cols = full_table.columns

x = 1
plt.figure(figsize=(18,4))
for col in cols:
    plt.subplot(1,5,x)
    plt.scatter(full_table[col], full_table["pts_diff"])
    plt.title(col)
    
    if (x == 5):
        x = 1
        plt.show()
        plt.figure(figsize=(18,4))
    else:
        x += 1

## Correlation Maps

In [ ]:
corr1 = full_table[num_cols + bin_cols]
corrmap1 = corr1.corr()

In [ ]:
corrmap1

In [ ]:
corr2 = full_table[cat_cols + ['pts_diff']]
corrmap2 = corr2.corr()

In [ ]:
corrmap2

## Heat Maps

In [ ]:
plt.figure(figsize=(15, 15))

sns.heatmap(corrmap1, square=True, annot=True, linewidths=.5)
plt.title("Correlation Matrix")

plt.show()

In [ ]:
plt.figure(figsize=(15, 15))

sns.heatmap(corrmap2, square=True, annot=True, linewidths=.5)
plt.title("Correlation Matrix")

plt.show()

In [ ]:
c_cols = ['d_winner', 'd_loser']
n_cols = ['score_home', 'score_away', 'away_odds_open', 'yds_diff', 'TO_diff']

In [ ]:
X = full_table[c_cols + n_cols]
y = full_table['pts_diff']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X

In [ ]:
preprocessing = ColumnTransformer([
    # Should only use one of these
    # Comment out or delete one of the below 2 lines
#    ('OneHotEncoder', OneHotEncoder(drop=drop_cats), cat_cols),    
    ('leaveoneoutencoder', LeaveOneOutEncoder(), c_cols),

    # Scale numeric columns (not needed for all models but can't hurt)
    ('scaler', StandardScaler(), n_cols)
    
    # bin_cols we'll leave untouch
], remainder='passthrough')

## Lasso

In [ ]:
pipeline = Pipeline([
    ('preprocessing', preprocessing),
    # Choose your model and put it here
    ('model', LassoCV())
])


grid = {
    # Use model__ with hyperprammeter names after
    'model__cv':[3, 5, 7],
    'model__n_jobs': [0.1, 1, 10]
}

pipeline_cv = GridSearchCV(pipeline, grid)
pipeline_cv.fit(X_train, y_train)

In [ ]:
pipeline_cv.best_params_

In [ ]:
print(pipeline_cv.score(X_train, y_train))
print(pipeline_cv.score(X_test, y_test))

## Ridge

In [ ]:
pipeline = Pipeline([
    ('preprocessing', preprocessing),
    # Choose your model and put it here
    ('model', RidgeCV())
])


grid = {
    # Use model__ with hyperprammeter names after
    'model__cv':[1, 3, 5]
}

pipeline_cv = GridSearchCV(pipeline, grid)
pipeline_cv.fit(X_train, y_train)

In [ ]:
pipeline_cv.best_params_

In [ ]:
print(pipeline_cv.score(X_train, y_train))
print(pipeline_cv.score(X_test, y_test))

## ElasticNet

In [ ]:
pipeline = Pipeline([
    ('preprocessing', preprocessing),
    # Choose your model and put it here
    ('model', ElasticNetCV())
])


grid = {
    # Use model__ with hyperprammeter names after
    "model__n_jobs": [0.1, 1, 10], 
    "model__cv": [3, 5, 7]
}

pipeline_cv = GridSearchCV(pipeline, grid)
pipeline_cv.fit(X_train, y_train)

In [ ]:
pipeline_cv.best_params_

In [ ]:
print(pipeline_cv.score(X_train, y_train))
print(pipeline_cv.score(X_test, y_test))

## Evaluate your best model on the test set

In [ ]:
model = RidgeCV(cv=3) 
model.fit(X_train, y_train)

In [ ]:
model.score(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

## Next, to prepare for your presentation, create a slide deck and a 15-minute presentation that guides viewers through your model. Be sure to cover a few specific topics:

* ## A specified research question that your model addresses
* ## How you chose your model specification and what alternatives you compared it to
* ## The practical uses of your model for an audience of interest
* ## Any weak points or shortcomings of your model

## This presentation is not a drill. You'll be presenting this slide deck live to a group as the culmination of all your work so far on supervised learning. As a secondary matter, your slides and the Jupyter Notebook should be worthy of inclusion as examples of your work product when applying to jobs.